In [3]:


#use rsync instead of scp
! rsync -avz -e "ssh -p 1992" briarmoss@briarmossdesktop:/home/briarmoss/Documents/Boston_311/models/cls_and_pooled_embeddings_with_service_id.csv .

receiving incremental file list
cls_and_pooled_embeddings_with_service_id.csv

sent 85,804 bytes  received 481,175,542 bytes  2,998,513.06 bytes/sec
total size is 1,277,269,840  speedup is 2.65


In [2]:
! scp -P 1992 briarmoss@briarmossdesktop:/home/briarmoss/Documents/Boston_311/models/Featrix_demo.html .

Featrix_demo.html                             100%  624KB   5.6MB/s   00:00    


In [ ]:
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp *_311_cases.csv {EXPORT_FOLDER}
!cp *_311_predictions.csv {EXPORT_FOLDER}
!cp *_311_ml_models.csv {EXPORT_FOLDER}
!cp *_manifest.txt {EXPORT_FOLDER}



** Copy the files to the production server **

In [ ]:
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

In [ ]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"*_311_cases.csv",
    f"*_311_predictions.csv",
    f"*_311_ml_models.csv",
    f"*_manifest.txt"
]

# Control where to copy
copy_to_prod = False
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


In [ ]:
PORT_NUMBER

In [ ]:
!ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

In [ ]:
!ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'